# 🚀 Fine-Tuning DeepSeek-R1-Distill-Qwen-1.5B on the apigen-80k Dataset

This project demonstrates the fine-tuning of the language model **[`deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B)** using the **`apigen-80k`** split of the [`HuggingFaceTB/smoltalk`](https://huggingface.co/datasets/HuggingFaceTB/smoltalk) dataset.

## 📌 Objective

To enhance the model's ability to handle instruction-style and chat-based tasks in developer-focused domains (e.g., API interactions), using **parameter-efficient fine-tuning (LoRA)** via the Hugging Face `peft` and `trl` libraries.

## 🧠 Model

- **Base Model:** `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B`
- **Architecture:** Qwen-compatible, instruction-tuned, lightweight (~1.5B parameters)
- **Fine-Tuning Method:** LoRA (Low-Rank Adaptation)

## 📚 Dataset

- **Dataset:** `HuggingFaceTB/smoltalk`
- **Split:** `apigen-80k`
- **Format:** Chat-style with `messages` as input (list of `{role, content}`)

## 🔧 Training Setup

- Framework: Hugging Face `transformers`, `peft`, `trl`
- LoRA Config: `r=6`, `alpha=8`, `dropout=0.05`
- Sequence Length: `1024`
- Strategy: `packing=True` for efficient token utilization

## ✅ Result

The final LoRA-adapted model can be merged and exported for deployment as a standard Hugging Face model.



## Dataset Preparation
The supervised fine-tuning process requires a task-specific dataset structured with input-output pairs. Each pair should consist of: <br>

1. An input prompt <br>
2. The expected model response <br>
3. Any additional context or metadata <br>



In [4]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
import torch


2025-06-06 20:23:57.765978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749241438.002929      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749241438.064668      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# pip install trl

In [5]:
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
# Load dataset
dataset = load_dataset("HuggingFaceTB/smoltalk", "apigen-80k")

README.md:   0%|          | 0.00/9.72k [00:00<?, ?B/s]

data/apigen-80k/train-00000-of-00001.par(…):   0%|          | 0.00/49.8M [00:00<?, ?B/s]

data/apigen-80k/test-00000-of-00001.parq(…):   0%|          | 0.00/2.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/83144 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4377 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 83144
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 4377
    })
})

### Import model and tokenizer

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [10]:
# Define chat template setup (required for chat-style data)
def setup_chat_format(model, tokenizer):
    tokenizer.chat_template = "{% for message in messages %}{{ message['role'] }}: {{ message['content'] }}\n{% endfor %}assistant: "
    return model, tokenizer

In [11]:
# Apply chat template
model, tokenizer = setup_chat_format(model, tokenizer)

## Set LoRA Adapters with PEFT
LoRA (Low-Rank Adaptation) is a parameter-efficient fine-tuning technique that freezes the pre-trained model weights and injects trainable rank decomposition matrices into the model’s layers.

In [11]:
# from peft import PeftModel, PeftConfig

# config = PeftConfig.from_pretrained("ybelkada/opt-350m-lora")
# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
# lora_model = PeftModel.from_pretrained(model, "ybelkada/opt-350m-lora")

In [12]:
from peft import LoraConfig

# TODO: Configure LoRA parameters
# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 6
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 8
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,  # Rank dimension - typically between 4-32
    lora_alpha=lora_alpha,  # LoRA scaling factor - typically 2x rank
    lora_dropout=lora_dropout,  # Dropout probability for LoRA layers
    bias="none",  # Bias type for LoRA. the corresponding biases will be updated during training.
    target_modules="all-linear",  # Which modules to apply LoRA to
    task_type="CAUSAL_LM",  # Task type for model architecture
)

## Training

In [13]:
# Training configuration
from trl import SFTConfig

training_args = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    eval_steps=50,
    max_seq_length=1024,  
    packing=True           # Optional: for better GPU utilization
)


In [14]:
training_args = SFTConfig(packing=True)
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    processing_class=tokenizer,
)


Converting train dataset to ChatML:   0%|          | 0/83144 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/83144 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/83144 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/83144 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [18]:
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 280.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 89.12 MiB is free. Process 5512 has 15.80 GiB memory in use. Of the allocated memory 15.31 GiB is allocated by PyTorch, and 201.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Merge and Save the Final Model
After training, merge the LoRA adapter into the base model and save it.

In [ ]:
from peft import PeftModel
from transformers import AutoTokenizer

# Step 1: Merge LoRA weights into base model
merged_model = model.merge_and_unload()  # model is your PEFT-wrapped model

# Step 2: Save the merged model and tokenizer
save_path = "./merged_model"
merged_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"✅ Merged model saved to {save_path}")


# Evaluation

In [ ]:
results = trainer.evaluate()
print(results)


In [ ]:
from datasets import load_metric

rouge = load_metric("rouge")

# Hypothetical comparison
scores = rouge.compute(predictions=[generated_text], references=[expected_answer])
print(scores)
